# CLIP

CLIP 模型是在 2021 年 OpenAI 在 [Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/abs/2103.00020) 提出。CLIP（Contrastive Language-Image Pre-Training）是一种在大量的图像，文本数据对上训练的神经网络。对于给定的图片，CLIP 可以直接预测和该图片最相关的文本描述，这样的能力赋予 CLIP 零样本的能力。

<div class="alert alert-info">
**摘要：**

当前最先进的计算机视觉系统通常被训练用于预测一组固定的预设物体类别。这种受限的监督形式限制了模型的通用性和可用性，因为需要额外标注的数据来定义其他视觉概念。而直接从与图像相关的原始文本中学习提供了一种有前景的替代方法，能够利用更广泛的监督来源。我们展示了一种简单但有效的预训练任务：预测哪段文字与哪张图像匹配。通过在互联网上收集的4亿（图像，文本）对数据集上从零开始进行训练，这种方法能够高效、可扩展地学习最先进的图像表示。预训练完成后，模型可以通过自然语言引用学习到的视觉概念（或描述新的概念），从而实现模型在下游任务中的零样本迁移。

我们通过在超过 30 个现有计算机视觉数据集上的基准测试，研究了这种方法的性能，涵盖任务包括OCR（光学字符识别）、视频中的动作识别、地理定位以及多种细粒度物体分类。模型在大多数任务中的迁移性能显著，且通常能够在无需任何特定数据集训练的情况下，与全监督的基线模型竞争。例如，我们在ImageNet上的零样本分类准确率与原始ResNet-50相当，而无需使用任何原本用于训练的128万训练样本。
</div>

## 在 🤗 Transformers 中使用 CLIP 模型

`CLIPTokenizer` 用于编码文本。`CLIPProcessor` 将 `CLIPImageProcessor` 和 `CLIPTokenizer` 封装成一个实例，以同时编码文本和准备图像。以下示例展示了如何使用 `CLIPProcessor` 和 `CLIPModel` 获取图像-文本相似度分数。

In [ ]:
from PIL import Image
import requests
import torch


from transformers import CLIPProcessor, CLIPModel

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32", torch_dtype=torch.bfloat16, device_map=device
)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)
inputs = inputs.to(device)

with torch.inference_mode():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(
    dim=1
)  # we can take the softmax to get the label probabilities
print(probs)

tensor([[0.9961, 0.0046]], device='cuda:0', dtype=torch.bfloat16)
